# **Install Dependencies**

In [1]:
!pip install -q fastapi uvicorn pyngrok gradio transformers torch torchaudio soundfile pydub
!pip install -q openai-whisper speechrecognition google-cloud-texttospeech gTTS
!pip install -q nest-asyncio python-multipart

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 43.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.2/192.2 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 9.1 MB/s eta 0:00:00


# **Import Libraries and Setup**

In [2]:
import os
import json
import asyncio
import nest_asyncio
from datetime import datetime, timedelta
from typing import Optional, List, Dict
import gradio as gr
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from pyngrok import ngrok
import uvicorn
from threading import Thread
import speech_recognition as sr
from gtts import gTTS
import tempfile
import re

nest_asyncio.apply()

# **Data Models and Dummy Database**

In [3]:
class Schedule(BaseModel):
    id: Optional[int] = None
    patient_name: str
    appointment_date: str
    appointment_time: str
    doctor: str
    reason: str
    status: str = "confirmed"

class AppointmentRequest(BaseModel):
    patient_name: str
    date: str
    time: str
    doctor: Optional[str] = "Dr. Smith"
    reason: Optional[str] = "General checkup"

# Dummy database - In-memory storage
schedules_db = [
    {
        "id": 1,
        "patient_name": "John Doe",
        "appointment_date": (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d"),
        "appointment_time": "10:00 AM",
        "doctor": "Dr. Smith",
        "reason": "Annual checkup",
        "status": "confirmed"
    },
    {
        "id": 2,
        "patient_name": "Jane Smith",
        "appointment_date": (datetime.now() + timedelta(days=2)).strftime("%Y-%m-%d"),
        "appointment_time": "2:00 PM",
        "doctor": "Dr. Johnson",
        "reason": "Follow-up",
        "status": "confirmed"
    },
    {
        "id": 3,
        "patient_name": "Bob Wilson",
        "appointment_date": (datetime.now() + timedelta(days=3)).strftime("%Y-%m-%d"),
        "appointment_time": "11:30 AM",
        "doctor": "Dr. Smith",
        "reason": "Consultation",
        "status": "confirmed"
    }
]

# Available time slots
available_slots = [
    "9:00 AM", "10:00 AM", "11:00 AM", "12:00 PM",
    "2:00 PM", "3:00 PM", "4:00 PM", "5:00 PM"
]


# **FastAPI Setup**

In [4]:
app = FastAPI(title="Voice Automation Agent API")

@app.get("/")
async def root():
    return {"message": "Voice Automation Agent API", "status": "running"}

@app.get("/schedules")
async def get_schedules():
    """Get all scheduled appointments"""
    return {"schedules": schedules_db, "count": len(schedules_db)}

@app.get("/schedules/upcoming")
async def get_upcoming_schedules():
    """Get upcoming appointments"""
    today = datetime.now().date()
    upcoming = [
        s for s in schedules_db
        if datetime.strptime(s["appointment_date"], "%Y-%m-%d").date() >= today
    ]
    return {"schedules": upcoming, "count": len(upcoming)}

@app.get("/available-slots")
async def get_available_slots(date: str):
    """Get available time slots for a specific date"""
    booked_slots = [
        s["appointment_time"] for s in schedules_db
        if s["appointment_date"] == date
    ]
    available = [slot for slot in available_slots if slot not in booked_slots]
    return {"date": date, "available_slots": available}

@app.post("/schedule")
async def create_schedule(appointment: AppointmentRequest):
    """Book a new appointment"""
    # Validate date format
    try:
        datetime.strptime(appointment.date, "%Y-%m-%d")
    except ValueError:
        raise HTTPException(status_code=400, detail="Invalid date format. Use YYYY-MM-DD")

    # Check if slot is available
    existing = next((s for s in schedules_db
                    if s["appointment_date"] == appointment.date
                    and s["appointment_time"] == appointment.time), None)

    if existing:
        raise HTTPException(status_code=409, detail="Time slot already booked")

    # Create new appointment
    new_id = max([s["id"] for s in schedules_db]) + 1 if schedules_db else 1
    new_schedule = {
        "id": new_id,
        "patient_name": appointment.patient_name,
        "appointment_date": appointment.date,
        "appointment_time": appointment.time,
        "doctor": appointment.doctor,
        "reason": appointment.reason,
        "status": "confirmed"
    }

    schedules_db.append(new_schedule)
    return {"message": "Appointment booked successfully", "appointment": new_schedule}

@app.delete("/schedules/{schedule_id}")
async def cancel_schedule(schedule_id: int):
    """Cancel an appointment"""
    global schedules_db
    schedule = next((s for s in schedules_db if s["id"] == schedule_id), None)

    if not schedule:
        raise HTTPException(status_code=404, detail="Schedule not found")

    schedules_db = [s for s in schedules_db if s["id"] != schedule_id]
    return {"message": "Appointment cancelled successfully", "cancelled": schedule}


# ***Voice Processing Functions***

In [5]:
def text_to_speech(text: str) -> str:
    """Convert text to speech and return audio file path"""
    tts = gTTS(text=text, lang='en', slow=False)
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.mp3')
    tts.save(temp_file.name)
    return temp_file.name

def speech_to_text(audio_file) -> str:
    """Convert speech to text using speech recognition"""
    recognizer = sr.Recognizer()

    try:
        with sr.AudioFile(audio_file) as source:
            audio_data = recognizer.record(source)
            text = recognizer.recognize_google(audio_data)
            return text
    except sr.UnknownValueError:
        return "Sorry, I could not understand the audio."
    except sr.RequestError as e:
        return f"Could not request results; {e}"
    except Exception as e:
        return f"Error: {str(e)}"


# **Natural Language Processing for Intent Detection**

In [22]:
def parse_intent(text: str) -> Dict:
    """Parse user intent from text"""
    text_lower = text.lower()

    # Check for listing schedules
    if any(word in text_lower for word in ['list', 'show', 'upcoming', 'schedules', 'appointments']):
        return {"intent": "list_schedules", "params": {}}

    # Check for booking
    if any(word in text_lower for word in ['book', 'schedule', 'appointment', 'make']):
        intent = {"intent": "book_appointment", "params": {}}

        # Extract name
        name_patterns = [
            r"(?:for|name is|i am|i'm)\s+([a-zA-Z\s]+?)(?:\s+on|\s+at|\s+for|$)",
            r"patient\s+([a-zA-Z\s]+?)(?:\s+on|\s+at|\s+for|$)"
        ]
        for pattern in name_patterns:
            match = re.search(pattern, text_lower)
            if match:
                intent["params"]["name"] = match.group(1).strip().title()
                break

        # Extract date
        date_patterns = [
            r"on\s+(tomorrow|today|next\s+\w+)",
            r"(\d{4}-\d{2}-\d{2})",
            r"(january|february|march|april|may|june|july|august|september|october|november|december)\s+(\d+)"
        ]
        for pattern in date_patterns:
            match = re.search(pattern, text_lower)
            if match:
                date_str = match.group(0)
                if "tomorrow" in date_str:
                    intent["params"]["date"] = (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d")
                elif "today" in date_str:
                    intent["params"]["date"] = datetime.now().strftime("%Y-%m-%d")
                break

        # Extract time
        # Matches: "10 a.m.", "10 AM", "10am", "10:30 p.m.", etc.
        time_patterns = [
            r"at\s+(\d+:?\d*\s*(?:a\.?m\.?|p\.?m\.?))",  # Matches a.m. and am with "at"
            r"(\d+:?\d*\s*(?:a\.?m\.?|p\.?m\.?))",       # More flexible without "at"
            r"at\s+(\d+:?\d*)\s*(?:o'clock|oclock)?",    # Handles o'clock
            r"(\d+)\s*(?:o'clock|oclock)"                # Just number + o'clock
        ]

        for pattern in time_patterns:
            match = re.search(pattern, text_lower)
            if match:
                time_str = match.group(1).strip()

                # Normalize a.m./p.m. variations
                time_str = time_str.replace('a.m.', 'AM').replace('a.m', 'AM')
                time_str = time_str.replace('p.m.', 'PM').replace('p.m', 'PM')
                time_str = time_str.replace('am', 'AM').replace('pm', 'PM')

                # Format time properly (add :00 if missing minutes)
                if ':' not in time_str and ('AM' in time_str or 'PM' in time_str):
                    parts = time_str.split()
                    if len(parts) >= 2:
                        time_str = parts[0] + ':00 ' + parts[-1]
                    else:
                        # Handle case like "10AM" with no space
                        for meridiem in ['AM', 'PM']:
                            if meridiem in time_str:
                                time_str = time_str.replace(meridiem, ':00 ' + meridiem)
                                break

                intent["params"]["time"] = time_str.upper()
                break

        return intent

    # Check for available slots
    if any(word in text_lower for word in ['available', 'free', 'slots', 'times']):
        return {"intent": "check_availability", "params": {}}

    return {"intent": "unknown", "params": {}}


# **Agent Logic**

In [23]:
class VoiceAgent:
    def __init__(self, api_url: str):
        self.api_url = api_url
        self.conversation_context = {
            "booking_in_progress": False,
            "booking_data": {}
        }

    async def process_voice_input(self, audio_file) -> tuple:
        """Process voice input and return response text and audio"""
        # Convert speech to text
        user_text = speech_to_text(audio_file)

        if "Error" in user_text or "Sorry" in user_text:
            response_text = user_text
            audio_path = text_to_speech(response_text)
            return user_text, response_text, audio_path

        # Parse intent
        intent_data = parse_intent(user_text)

        # Process based on intent
        response_text = await self.handle_intent(intent_data, user_text)

        # Convert response to speech
        audio_path = text_to_speech(response_text)

        return user_text, response_text, audio_path

    async def handle_intent(self, intent_data: Dict, original_text: str) -> str:
        """Handle different intents"""
        intent = intent_data["intent"]
        params = intent_data["params"]

        # Check if we're in the middle of booking
        if self.conversation_context.get("booking_in_progress"):
            return await self.continue_booking(original_text)

        if intent == "list_schedules":
            return await self.list_schedules()

        elif intent == "book_appointment":
            return await self.start_booking(params, original_text)

        elif intent == "check_availability":
            return await self.check_availability()

        else:
            return ("I can help you with:\n"
                   "1. Listing upcoming appointments - say 'show my schedules'\n"
                   "2. Booking new appointments - say 'book' or 'book appointment'\n"
                   "3. Checking available slots - say 'show available times'")

    async def list_schedules(self) -> str:
        """List all upcoming schedules"""
        import aiohttp

        try:
            async with aiohttp.ClientSession() as session:
                async with session.get(f"{self.api_url}/schedules/upcoming") as resp:
                    data = await resp.json()
                    schedules = data["schedules"]

                    if not schedules:
                        return "There are no upcoming appointments scheduled."

                    response = f"You have {len(schedules)} upcoming appointments:\n\n"
                    for idx, s in enumerate(schedules, 1):
                        response += (f"{idx}. {s['patient_name']} with {s['doctor']} "
                                   f"on {s['appointment_date']} at {s['appointment_time']} "
                                   f"for {s['reason']}.\n")

                    return response
        except Exception as e:
            return f"Error fetching schedules: {str(e)}"

    async def start_booking(self, params: Dict, original_text: str) -> str:
        """Start the booking process - can be interactive"""
        # If user said just "book", start interactive mode
        if not params or len(params) == 0:
            self.conversation_context["booking_in_progress"] = True
            self.conversation_context["booking_data"] = {}
            return "Sure! I'll help you book an appointment. What is the patient's name?"

        # If user provided some info but not all, collect missing info
        self.conversation_context["booking_data"] = params.copy()

        required = ["name", "date", "time"]
        missing = [r for r in required if r not in params]

        if missing:
            self.conversation_context["booking_in_progress"] = True
            if "name" not in params:
                return "What is the patient's name?"
            elif "date" not in params:
                return f"Great! For {params['name']}, what date would you like? You can say 'tomorrow', 'today', or a specific date."
            elif "time" not in params:
                return f"What time works for {params['name']} on {params.get('date', 'that day')}?"

        # If all info provided, book immediately
        return await self.book_appointment(params, original_text)

    async def continue_booking(self, user_text: str) -> str:
        """Continue an in-progress booking conversation"""
        booking_data = self.conversation_context["booking_data"]
        text_lower = user_text.lower()

        # Check for cancel command
        if "cancel" in text_lower or "nevermind" in text_lower or "stop" in text_lower:
            self.conversation_context["booking_in_progress"] = False
            self.conversation_context["booking_data"] = {}
            return "Booking cancelled. How else can I help you?"

        # Collect patient name
        if "name" not in booking_data:
            # Extract name from response
            words = user_text.strip().split()
            if len(words) >= 1:
                booking_data["name"] = user_text.strip().title()
                self.conversation_context["booking_data"] = booking_data
                return f"Perfect! For {booking_data['name']}, what date would you like? You can say 'tomorrow', 'today', or a specific date."
            else:
                return "I didn't catch that. What is the patient's name?"

        # Collect date
        elif "date" not in booking_data:
            # Parse date from response
            if "tomorrow" in text_lower:
                booking_data["date"] = (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d")
            elif "today" in text_lower:
                booking_data["date"] = datetime.now().strftime("%Y-%m-%d")
            else:
                # Try to extract date pattern
                date_match = re.search(r'(\d{4}-\d{2}-\d{2})', text_lower)
                if date_match:
                    booking_data["date"] = date_match.group(1)
                else:
                    # Default to tomorrow if unclear
                    booking_data["date"] = (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d")

            self.conversation_context["booking_data"] = booking_data
            return f"Got it! {booking_data['date']}. What time would be good?"

        # Collect time
        elif "time" not in booking_data:
            # Extract time - matches all formats including a.m./p.m.
            time_match = re.search(r'(\d+:?\d*)\s*(?:a\.?m\.?|p\.?m\.?)', text_lower)

            if time_match:
                time_str = time_match.group(0).strip()

                # Normalize a.m./p.m. variations
                time_str = time_str.replace('a.m.', 'AM').replace('a.m', 'AM')
                time_str = time_str.replace('p.m.', 'PM').replace('p.m', 'PM')
                time_str = time_str.replace('am', 'AM').replace('pm', 'PM')

                # Better normalization and formatting
                if ':' not in time_str and ('AM' in time_str or 'PM' in time_str):
                    parts = time_str.split()
                    if len(parts) >= 2:
                        time_str = parts[0] + ':00 ' + parts[-1]
                    else:
                        # Handle "10AM" format
                        for meridiem in ['AM', 'PM']:
                            if meridiem in time_str:
                                time_str = time_str.replace(meridiem, ':00 ' + meridiem)
                                break

                booking_data["time"] = time_str.upper()
            else:
                # Try to extract just number + am/pm
                time_match = re.search(r'(\d+)\s*(am|pm)', text_lower)
                if time_match:
                    booking_data["time"] = f"{time_match.group(1)}:00 {time_match.group(2).upper()}"
                else:
                    return "I didn't catch the time. Please say a time like '10 AM' or '2 PM'."

            self.conversation_context["booking_data"] = booking_data

            # Check if we want to collect doctor preference
            if "doctor" not in booking_data:
                return "Which doctor would you prefer? Dr. Smith or Dr. Johnson? Or say 'any doctor'."

        # Collect doctor preference
        elif "doctor" not in booking_data:
            if "smith" in text_lower:
                booking_data["doctor"] = "Dr. Smith"
            elif "johnson" in text_lower:
                booking_data["doctor"] = "Dr. Johnson"
            else:
                booking_data["doctor"] = "Dr. Smith"  # Default

            self.conversation_context["booking_data"] = booking_data
            return "What's the reason for the visit? Or say 'general checkup'."

        # Collect reason
        elif "reason" not in booking_data:
            if len(user_text.strip()) > 0:
                booking_data["reason"] = user_text.strip()
            else:
                booking_data["reason"] = "General consultation"

            self.conversation_context["booking_data"] = booking_data

            # Now we have everything - book it!
            return await self.book_appointment(booking_data, user_text)

    async def book_appointment(self, params: Dict, original_text: str) -> str:
        """Book a new appointment"""
        import aiohttp

        # Check if we have all required information
        required = ["name", "date", "time"]
        missing = [r for r in required if r not in params]

        if missing:
            return (f"I need more information to book the appointment. "
                   f"Please provide: {', '.join(missing)}. "
                   f"For example: 'Book appointment for John Doe on tomorrow at 10 AM'")

        try:
            appointment_data = {
                "patient_name": params["name"],
                "date": params["date"],
                "time": params["time"],
                "doctor": params.get("doctor", "Dr. Smith"),
                "reason": params.get("reason", "General consultation")
            }

            async with aiohttp.ClientSession() as session:
                async with session.post(
                    f"{self.api_url}/schedule",
                    json=appointment_data
                ) as resp:
                    if resp.status == 200:
                        data = await resp.json()
                        appt = data["appointment"]

                        # Reset booking context
                        self.conversation_context["booking_in_progress"] = False
                        self.conversation_context["booking_data"] = {}

                        return (f"Great! I've successfully booked an appointment for "
                               f"{appt['patient_name']} with {appt['doctor']} "
                               f"on {appt['appointment_date']} at {appt['appointment_time']}. "
                               f"Your appointment ID is {appt['id']}.")
                    else:
                        error_data = await resp.json()

                        # Reset booking context on error
                        self.conversation_context["booking_in_progress"] = False
                        self.conversation_context["booking_data"] = {}

                        return f"Sorry, I couldn't book the appointment. {error_data.get('detail', 'Unknown error')}"

        except Exception as e:
            # Reset booking context on exception
            self.conversation_context["booking_in_progress"] = False
            self.conversation_context["booking_data"] = {}

            return f"Error booking appointment: {str(e)}"

    async def check_availability(self) -> str:
        """Check available slots"""
        tomorrow = (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d")
        import aiohttp

        try:
            async with aiohttp.ClientSession() as session:
                async with session.get(
                    f"{self.api_url}/available-slots?date={tomorrow}"
                ) as resp:
                    data = await resp.json()
                    slots = data["available_slots"]

                    if not slots:
                        return f"Sorry, there are no available slots for {tomorrow}."

                    return (f"Available time slots for {tomorrow}:\n" +
                           ", ".join(slots))
        except Exception as e:
            return f"Error checking availability: {str(e)}"

# **Gradio Interface**

In [24]:
def create_gradio_interface(agent: VoiceAgent):
    """Create Gradio interface for voice interaction"""

    async def process_audio(audio):
        if audio is None:
            return "Please record some audio first.", "", None

        user_text, response_text, audio_path = await agent.process_voice_input(audio)
        return user_text, response_text, audio_path

    with gr.Blocks(title="Voice Automation Agent", theme=gr.themes.Soft()) as demo:
        gr.Markdown(
            """
            # 🎤 Voice Automation Agent
            ### Hospital Appointment Scheduling System

            **Features:**
            - 📋 List upcoming appointments
            - 📅 Book new appointments via voice
            - ⏰ Check available time slots

            **Try saying:**
            - "Show me upcoming appointments"
            - "Book appointment for John Doe tomorrow at 10 AM"
            - "What time slots are available?"
            """
        )

        with gr.Row():
            with gr.Column(scale=1):
                audio_input = gr.Audio(
                    sources=["microphone"],
                    type="filepath",
                    label="🎙️ Speak your request"
                )
                submit_btn = gr.Button("Process Voice", variant="primary", size="lg")

            with gr.Column(scale=1):
                user_text_output = gr.Textbox(
                    label="📝 What you said",
                    lines=3
                )
                response_text_output = gr.Textbox(
                    label="💬 Agent Response",
                    lines=6
                )
                audio_output = gr.Audio(
                    label="🔊 Voice Response",
                    type="filepath"
                )

        gr.Markdown(
            """
            ### 📖 Example Commands:
            - "List my appointments"
            - "Show upcoming schedules"
            - "Book appointment for Jane Smith on tomorrow at 2 PM"
            - "What slots are available?"
            """
        )

        submit_btn.click(
            fn=process_audio,
            inputs=[audio_input],
            outputs=[user_text_output, response_text_output, audio_output]
        )

    return demo


# **Run Everything**

In [25]:
def run_server():
    """Run FastAPI server"""
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

# Start FastAPI in background thread
server_thread = Thread(target=run_server, daemon=True)
server_thread.start()

# Wait for server to start
import time
time.sleep(3)

# Setup ngrok tunnel
ngrok.set_auth_token("34qxjCcYJHm9we81c4jN3PZyMTL_4nQrowPTGv8oLoxZniB32")  # Replace with your ngrok token
public_url = ngrok.connect(8000)
api_url = public_url.public_url

print(f"✅ FastAPI Server running at: {api_url}")
print(f"📚 API Documentation: {api_url}/docs")
print(f"📊 View Schedules: {api_url}/schedules")

# Create agent
agent = VoiceAgent(api_url)

# Create and launch Gradio interface
demo = create_gradio_interface(agent)
demo.launch(share=True, debug=True)


INFO:     Started server process [208]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): [errno 98] address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-737' coro=<Server.serve() done, defined at /usr/local/lib/python3.12/dist-packages/uvicorn/server.py:69> exception=SystemExit(1)>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/server.py", line 164, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/asyncio/base_events.py", line 1584, in create_server
    raise OSError(err.errno, msg) from None
OSError: [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): [errno 98] address already in use

During handling of th

✅ FastAPI Server running at: https://waspier-rowen-semialcoholic.ngrok-free.dev
📚 API Documentation: https://waspier-rowen-semialcoholic.ngrok-free.dev/docs
📊 View Schedules: https://waspier-rowen-semialcoholic.ngrok-free.dev/schedules
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://499d625eae165ff48e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://499d625eae165ff48e.gradio.live
